In [ ]:
#可跑的參數TS在最下面

In [1]:
import numpy as np
A = []
f = open('att48_xy.txt')        #開啟資料檔案檔案
lines = f.readlines()      #把全部資料檔案讀到一個列表lines中
ind = 0
for line in lines:       #把lines中的資料逐行讀取出來
    lista = line.strip('\n').split(' ')   #處理逐行資料：strip表示把頭尾的'\n'去掉，split表示以空格來分割行資料，然後把處理後的行資料返回到list列表中
    lista = [i for i in lista if(len(str(i))!=0)]
    A.append([int(lista[0]), int(lista[1])])          #把處理後的資料放到方陣A中。
    ind += 1
#print(A)

def create_data(N):
    calDistance = lambda x, y: np.sqrt((x[0] - y[0]) ** 2 + (x[1] - y[1]) ** 2)
    points = A
    Mat = np.zeros((N, N))
    data = []

    for i in range(N):
        for j in range(i + 1, N):
            dv = calDistance(points[i], points[j])
            Mat[i][j], Mat[j][i] = dv, dv
            data.append([i,j, dv])
            data.append([j,i, dv])
    return data

In [2]:
import math
from random import randint
import time
from random import shuffle
import numpy as np

###  Data Format is dict: 
#           data[node_name] = gives you a list of link info
#           data[link_index][0] = name of node that edge goes to  
#           data[link_index][1] = weight of that edge
def read_data():
    linkset = []
    links = {}
    max_weight = 0
    
    tmp = create_data(48)
    for i in range(len(tmp)):
        link = []
        link.append(tmp[i][0])
        link.append(tmp[i][1])
        link.append(int(tmp[i][2]))
        linkset.append(link)
            
        if int(tmp[i][2]) > max_weight:
            max_weight = int(tmp[i][2])
            
    
    for link in linkset:
        try:
            linklist = links[str(link[0])]
            linklist.append(link[1:])
            links[str(link[0])] = linklist
        except:
            links[str(link[0])] = [link[1:]]
        
    return links, max_weight

In [3]:
import math
from random import randint
import time
from random import shuffle

###  Data Format is dict: 
#           data[node_name] = gives you a list of link info
#           data[link_index][0] = name of node that edge goes to  
#           data[link_index][1] = weight of that edge


def getNeighbors(state):
    #return hill_climbing(state)
    return two_opt_swap(state)
        
def hill_climbing(state):
    node = randint(1, len(state)-1)
    neighbors = []
    
    for i in range(len(state)):
        if i != node and i != 0:
            tmp_state = state.copy()
            tmp = tmp_state[i]
            tmp_state[i] = tmp_state[node]
            tmp_state[node] = tmp
            neighbors.append(tmp_state)
            
    return neighbors

def two_opt_swap(state):
    global neighborhood_size
    neighbors = []
    
    for i in range(neighborhood_size):
        node1 = 0
        node2 = 0
        
        while node1 == node2:
            node1 = randint(1, len(state)-1)
            node2 = randint(1, len(state)-1)
            
        if node1 > node2:
            swap = node1
            node1 = node2
            node2 = swap
            
        
        tmp = state[node1:node2]
        tmp_state = state[:node1] + tmp[::-1] +state[node2:]
        neighbors.append(tmp_state)
        
    return neighbors

def fitness(route, graph):
    path_length = 0
    
    for i in range(len(route)):
        if(i+1 != len(route)):
            dist = weight_distance(route[i], route[i+1], graph)
            if dist != -1:
                path_length = path_length + dist
            else:
                return max_fitness # there is no  such path
                
        else:
            dist = weight_distance(route[i], route[0], graph)
            if dist != -1:
                path_length = path_length + dist
            else:
                return max_fitness # there is no  such path
            
    return path_length
            

def weight_distance(city1, city2, graph):
    global max_fitness
    
    neighbors = graph[str(city1)]
    
    for neighbor in neighbors:
        if neighbor[0] == int(city2):
            return neighbor[1]
        
    return -1 #there can't be minus distance, so -1 means there is not any city found in graph or there is not such edge


def tabu_search():
    global max_fitness, start_node
    graph, max_weight = read_data()
    
    ## Below, get the keys (node names) and shuffle them, and make start_node as start
    s0 = list(graph.keys())
    shuffle(s0)
    
    if int(s0[0]) != start_node:
        for i in range(len(s0)):
            if  int(s0[i]) == start_node:
                swap = s0[0]
                s0[0] = s0[i]
                s0[i] = swap
                break;
    
    # max_fitness will act like infinite fitness
    max_fitness = ((max_weight) * (len(s0)))+1
    sBest = s0
    vBest = fitness(s0, graph)
    bestCandidate = s0
    tabuList = []
    tabuList.append(s0)
    stop = False
    best_keep_turn = 0
    
    start_time = time.time()
    while not stop :
        sNeighborhood = getNeighbors(bestCandidate)
        bestCandidate = sNeighborhood[0]
        for sCandidate in sNeighborhood:
            if (sCandidate not in tabuList) and ((fitness(sCandidate, graph) < fitness(bestCandidate, graph))):
                bestCandidate = sCandidate

        if (fitness(bestCandidate, graph) < fitness(sBest, graph)):
            sBest = bestCandidate
            vBest = fitness(sBest, graph)
            best_keep_turn = 0

        tabuList.append(bestCandidate)
        if (len(tabuList) > maxTabuSize):
            tabuList.pop(0)
            
        if best_keep_turn == stoppingTurn:
            stop = True
            
        best_keep_turn += 1
        
    exec_time =  time.time() - start_time
    return sBest, vBest, exec_time
    


## Tabu Search Takes edge-list in a given format:
#nodefrom nodeto weight
#0 1 5
#3 2 4
#1 0 3
#Undirectional edges should be written 2 times for both nodes.



In [ ]:
totalv=[]
totalt=[]
for i in range(0,10):   
        maxTabuSize = 10
        neighborhood_size = 100
        stoppingTurn = 150
        max_fitness = 0
        start_node = 0

        solution, value, exec_time = tabu_search()
        totalv.append(value)
        totalt.append(exec_time)

        print(value)
        print(exec_time)
print(sum(totalv)/len(totalv))
print(sum(totalt)/len(totalt))

In [4]:
totalv=[]
totalt=[]
for i in range(0,10):   
        maxTabuSize = 100
        neighborhood_size = 150
        stoppingTurn = 1000
        max_fitness = 0
        start_node = 0
        solution, value, exec_time = tabu_search()
        totalv.append(value)
        totalt.append(exec_time)

        print(value)
        print(exec_time)
print(sum(totalv)/len(totalv))
print(sum(totalt)/len(totalt))

33888
152.98010659217834
35842
152.0007679462433
34343
247.76172280311584
35399
275.86293959617615
34099
138.58408069610596
33764
191.22031021118164
35602
349.8247437477112
34972
148.53902435302734
36661
175.61510491371155
36955
170.46767663955688
35152.5
200.28564774990082
